# Demo de Plotly Dash
## Diploma en Data Science - Instituto CPE
### Rafael Xavier

Plotly Dash es un paquete de Python (también de R y Julia) para la creación de webapps enfocadas en análisis de datos. Es un contexto de alto nivel, en el que el usuario no necesita saber demasiado de lo que está pasando detrás de escenas, y que permite crear interfaces avanzadas con facilidad. Ni siquiera es necesario saber demasiado de HTML y CSS.

Su uso más difundido es el crear dashboards que permitan resumir un gran número de datos de manera visualmente útil.

Este demo va a usar solamente `Pandas` (ya incluido en Colab) y `Dash`. 
Como estamos en un entorno de notebook, tenemos que usar `JupyterDash` en lugar de `Dash`, pero la funcionalidad es idéntica.

In [1]:
%%capture
!pip install jupyter-dash
!pip install pandas

import pandas as pd
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
from IPython.display import HTML

El dataset que vamos a usar es inventado. Representa las ventas de un comercio con sucursales en todo el país, e incluye el tipo de producto, precio unitario, tasa de IVA, departamento de la venta, etc.

In [2]:
data_url = "https://raw.githubusercontent.com/rxavier/cpe-demo/main/sample_data.csv"
df = pd.read_csv(data_url, index_col="fecha")
df.index = pd.to_datetime(df.index)
HTML(df.head().to_html())

,id,producto,unidades,precio,subtotal,tasa_iva,iva,total,departamento
fecha,,,,,,,,,
2019-05-23,1,Limpieza,8,340,2720,0%,0.00,2720.00,Canelones
2019-04-14,2,Alimentos,7,308,2156,10%,215.60,2371.60,Florida
2019-04-16,3,Bebidas,4,297,1188,22%,261.36,1449.36,Cerro Largo
2019-04-29,4,Bebidas,7,419,2933,0%,0.00,2933.00,Artigas
2020-01-02,5,Higiene,3,327,981,10%,98.10,1079.10,Durazno


A continuación definimos el setup básico de una app de Dash. Primero hago referencia a una hoja de estilo de CSS externa creada por el desarrollador de Dash. Esto es opcional pero deja la app bastante más atractiva.

Lo siguiente es definir la app propiamente dicha (si no estuviéramos en un entorno de notebook pondríamos `Dash` en lugar de `JupyterDash`) y a continuación definir el layout, es decir, los distintos componentes HTML que van a conformar la webapp.

Cabe destacar que este layout no tiene por qué ser el final, ya que como vamos a ver más adelante se puede seguir extendiendo, e incluso puede ser modificable por el usuario.

Como verán, no escribimos HTML propiamente dicho, sino que estamos haciendo referencia a clases de `dash_html_dependencies`. Este módulo tiene todos los componentes de HTML que usualmente querríamos usar, como `div`, `p`, `br`, `h1`, etc.

Con esas pocas líneas ya tenemos una interfaz web para chequear. Para ello, corremos la app.

No es mucho, pero sabemos que funciona. 

In [3]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([html.H1("Demo de Plotly Dash"),
                       html.H2("Diploma en Data Science - Instituto CPE"),
                       html.H3("Rafael Xavier")])

app.run_server(mode="external")

Dash app running on:


<IPython.core.display.Javascript object>

Lo siguiente es incorporar un gráfico simple.

Este gráfico, como todos los de Dash por defecto, es interactivo: muestra los valores al pasar el cursor, permite hacer zoom e incluso descargarlo como imagen.

En este primer paso simplemente vamos a acumular mensualmente las ventas de este comercio y mostrarlo en un gráfico de líneas. Para ello vamos a usar el paquete `Plotly Express`, que está incluído en `Dash`, y facilita enormemente la creación de gráficos razonablemente complejos.

Además vamos a extender el layout ya definido para agregar un elemento `Graph` del módulo `dash_core_components`, que funciona como contenedor de la gráfica propiamente dicha.

Con el layout ya actualizado vamos a reiniciar la app, que va a actualizar automáticamente la interfaz para incluir el gráfico.

In [4]:
data = df.resample("M").sum()
primera_grafica = px.line(data, y="total", x=data.index,
                          title="Ventas totales mensuales",
                          color_discrete_sequence=px.colors.qualitative.Vivid,
                          template="plotly_white")

simple = [html.H2("Gráfica simple sin callbacks"),
          dcc.Graph(id="primera", figure=primera_grafica)]

app.layout.children.extend(simple)

app.run_server(mode="external")

Dash app running on:


<IPython.core.display.Javascript object>

En un siguiente paso vamos a permitir que el usuario pueda elegir qué productos mostrar en la gráfica. Este tipo de funcionalidades es útil para darle la oportunidad a miembros del equipo de conocer los datos de manera iterativa, sin abrumar con decenas de gráficos distintos.

En `Dash` esto se logra a través de `callbacks`, que siguen la siguiente lógica: decime qué atributo de qué elemento querés que monitoree y decime de qué manera querés que lo procese. Luego de procesado, decime en qué atributo de qué elemento querés que ponga el resultado.

Esto que luce muy confuso va a quedar más claro en la app. Así que corremos la celda que extiende el layout y define el `callback`, y reiniciamos la app.

In [5]:
multifilter = [html.H2("Gráfica dinámica a través de callbacks."),
               dcc.Dropdown(
                   id="dropdown-multi-tipo",
                   options=[{"label": "Alimentos", "value": "Alimentos"},
                            {"label": "Bebidas", "value": "Bebidas"},
                            {"label": "Higiene", "value": "Higiene"},
                            {"label": "Limpieza", "value": "Limpieza"},
                            {"label": "Otros", "value": "Otros"}],
                   value=["Alimentos", "Higiene"], multi=True),
               dcc.Graph(id="segunda")]

app.layout.children.extend(multifilter)

@app.callback(
    Output("segunda", "figure"),
    [Input("dropdown-multi-tipo", "value")]
)
def segunda(tipos):
  if tipos is None or tipos == []:
    data = df
  else:
    data = df.loc[df.producto.isin(tipos)]
  data = (data.groupby("producto").resample("M").sum().
          reset_index().set_index("fecha"))
  fig = px.area(data, x=data.index, y="total", 
                color="producto", line_group="producto", 
                color_discrete_sequence=px.colors.qualitative.Vivid,
                title="Ventas totales por tipo", template="plotly_white")
  return fig

  app.run_server(mode="external")

Siguiendo la lógica anterior, lo que hicimos fue definir una lista desplegable (un `dash_core_components.Dropdown`) y le indicamos a la app que monitoree su valor. Ese valor es usado para filtrar el dataframe de los datos, y posteriormente crear el gráfico que es usado en el nuevo elemento `dash_core_components.Graph` que acabamos de definir.

Un `callback` puede tener varios inputs y varios outputs, abriendo enormemente las posibilidades para crear dashboards interactivos.

Hasta ahora vimos cómo crear gráficos dinámicos. Pero en `Dash` todo tiene el potencial de ser dinámico, ya que un `callback` puede tener como resultado la creación de más elementos HTML o la modificación de los existentes.

En el siguiente paso hacemos exactamente eso. El usuario podrá elegir qué filtros aplicar a los datos y qué valores toman esos filtros. Logramos esto mostrando y ocultando los filtros según si el usuario los seleccion, y actualizando el gráfico al mismo tiempo. Esto implica la creación de dos `callbacks`: uno que muestra y oculta los filtros, y otro que toma los valores de los filtros visibles y crea el gráfico.

In [6]:
dynamic = [html.H2("Interfaz dinámica a través de callbacks."),
           html.P("Definir filtros:"),
           dcc.Checklist(
               id="checklist",
               options=[{"label": "Tipo", "value": "tipo"},
                        {"label": "Precio", "value": "precio"},
                        {"label": "Unidades", "value": "unidades"},
                        {"label": "IVA", "value": "iva"},
                        {"label": "Departamento", "value": "departamento"}]),
           html.Br(),
           html.Div(id="filtros", children=[
               html.Div(id="tipo-container",
                        style={"display": "none"},
                        children=[html.Br(),
                                  dcc.Dropdown(
                                      id="tipo",
                                      options=[{"label": "Alimentos",
                                                "value": "Alimentos"},
                                               {"label": "Bebidas",
                                                "value": "Bebidas"},
                                               {"label": "Higiene",
                                                "value": "Higiene"},
                                               {"label": "Limpieza",
                                                "value": "Limpieza"},
                                               {"label": "Otros",
                                                "value": "Otros"}],
                                      multi=True)]),
               html.Div(id="precio-container",
                        style={"display": "none"},
                        children=[html.Br(),
                                  dcc.RangeSlider(id="precio",
                                                  min=10, max=500, step=1,
                                                  marks={0: "0", 100: "100",
                                                         200: "200",
                                                         300: "300",
                                                         400: "400",
                                                         500: "500"})]),
               html.Div(id="unidades-container",
                        style={"display": "none"},
                        children=[html.Br(),
                                  dcc.RangeSlider(id="unidades",
                                                  min=1, max=10, step=1,
                                                  marks={1: "1", 5: "5",
                                                         10: "10"})]),
               html.Div(id="iva-container",
                        style={"display": "none"},
                        children=[html.Br(), dcc.Dropdown(
                            id="iva",
                            options=[{"label": "22%", "value": "22%"},
                                     {"label": "10%", "value": "10%"},
                                     {"label": "0%", "value": "0%"}],
                            multi=True)]),
               html.Div(id="departamento-container",
                        style={"display": "none"},
                        children=[html.Br(), dcc.Dropdown(
                            id="departamento",
                            options=[{"label": "Artigas", "value": "Artigas"},
                                     {"label": "Canelones",
                                      "value": "Canelones"},
                                     {"label": "Cerro Largo",
                                      "value": "Cerro Largo"},
                                     {"label": "Colonia", "value": "Colonia"},
                                     {"label": "Durazno", "value": "Durazno"},
                                     {"label": "Flores", "value": "Flores"},
                                     {"label": "Florida", "value": "Florida"},
                                     {"label": "Lavalleja",
                                      "value": "Lavalleja"},
                                     {"label": "Maldonado",
                                      "value": "Maldonado"},
                                     {"label": "Montevideo",
                                      "value": "Montevideo"},
                                     {"label": "Paysandú",
                                      "value": "Paysandú"},
                                     {"label": "Río Negro",
                                      "value": "Río Negro"},
                                     {"label": "Rivera", "value": "Rivera"},
                                     {"label": "Rocha", "value": "Rocha"},
                                     {"label": "Salto", "value": "Salto"},
                                     {"label": "San José",
                                      "value": "San José"},
                                     {"label": "Soriano", "value": "Soriano"},
                                     {"label": "Tacuarembó",
                                      "value": "Tacuarembó"},
                                     {"label": "Treinta y Tres",
                                      "value": "Treinta y Tres"}],
                            multi=True)]),
               dcc.Graph(id="tercera")])]

app.layout.children.extend(dynamic)

@app.callback(
    [Output("tipo-container", "style"),
     Output("precio-container", "style"),
     Output("unidades-container", "style"),
     Output("iva-container", "style"),
     Output("departamento-container", "style")],
    [Input("checklist", "value")]
)
def definir_filtros(seleccion):
    if seleccion is None:
        return [{"display": "none"}, {"display": "none"}, {"display": "none"},
                {"display": "none"}, {"display": "none"}]
    styles = []
    for filtro in ["tipo", "precio", "unidades", "iva", "departamento"]:
        if filtro in seleccion:
            styles.append({"display": "block"})
        else:
            styles.append({"display": "none"})

    return styles

@app.callback(
    Output("tercera", "figure"),
    [Input("tipo", "value"),
     Input("precio", "value"),
     Input("unidades", "value"),
     Input("iva", "value"),
     Input("departamento", "value"),
     Input("tipo-container", "style"),
     Input("precio-container", "style"),
     Input("unidades-container", "style"),
     Input("iva-container", "style"),
     Input("departamento-container", "style")]
    )
def tercera(tipo, precio, unidades, tasa_iva, departamento,
            tipo_style, precio_style, unidades_style, iva_style, 
            departamento_style):
  data = df.copy()
  if tipo_style == {"display": "block"}:
      if tipo != [] and tipo is not None:
        data = data.loc[data["producto"].isin(tipo)]
  if precio_style == {"display": "block"}:
      if precio != [] and precio is not None:
        data = data.loc[data["precio"].between(precio[0], precio[1])]
  if unidades_style == {"display": "block"}:
      if unidades != [] and unidades is not None:
        data = data.loc[data["unidades"].between(unidades[0], unidades[1])]
  if iva_style == {"display": "block"}:
      if tasa_iva != [] and tasa_iva is not None:
        data = data.loc[data["tasa_iva"].isin(tasa_iva)]
  if departamento_style == {"display": "block"}:
      if departamento != [] and departamento is not None:
        data = data.loc[data["departamento"].isin(departamento)]
  data = data.resample("M").sum()
  fig = px.line(data, x=data.index, y="total",
                color_discrete_sequence=px.colors.qualitative.Vivid,
                title="Ventas totales", template="plotly_white")
  return fig

  app.run_server(mode="external")

Con esto estamos cubriendo lo básico de `Dash`, pero ya es suficiente para crear dashboards de alto valor en poco tiempo. Para ver qué otras posibilidades hay pueden chequear la [galería de apps de Dash en su web](https://dash-gallery.plotly.host/Portal/). Las secciones de [visualización](https://www.econ.uy/viz) y el [vistazo](https://www.econ.uy/viz) de mi sitio web econuy también fueron creadas con `Dash`.